In [ ]:
import polars as pl
import networkx as nx
import rtsvg
from linknode_graph_patterns import LinkNodeGraphPatterns
rt = rtsvg.RACETrack()
def graphAsDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _e_ in _g_.edges: _lu_['fm'].append(_e_[0]), _lu_['to'].append(_e_[1])
    return pl.DataFrame(_lu_)
_patterns_ = LinkNodeGraphPatterns()
_tiles_    = []
for _type_ in _patterns_.types:
    g   = _patterns_.createPattern(_type_)
    pos = nx.spring_layout(g)
    _tiles_.append(rt.link(graphAsDataFrame(g), [('fm', 'to')], pos=pos, w=256, h=256, node_size='small', link_arrow=False))
rt.tile(_tiles_, spacer=10)